In [ ]:
#@title Kaggle API

from IPython.display import clear_output

username = str(input("username: "))
key = str(input("key: "))

clear_output()

f = open("kaggle.json", "w")
f.write('{"username":"'+username+'","key":"'+key+'"}')
f.close()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d hernan4444/anime-recommendation-database-2020

import os, re
from zipfile import ZipFile

with ZipFile('/content/anime-recommendation-database-2020.zip', 'r') as zipObj:
   zipObj.extractall()

def purge(dir, pattern):
    for f in os.listdir(dir):
        if pattern in f:
            os.remove(os.path.join(dir, f))

purge("/content/", ".zip")
purge("/content/", ".json")

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Data Preprocessing

In [2]:
ratings = pd.read_csv("/content/animelist.csv")

In [3]:
ratings.head(5)

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,67,9,1,1
1,0,6702,7,1,4
2,0,242,10,1,4
3,0,4898,0,1,1
4,0,21,10,1,0


In the dataset, if a person hasn't left a rating to an anime, it was marked as a zero. Let's remove the zeros, as they will just introduce noise later on. Might as well remove the `watched_episodes`.

In [4]:
ratings = ratings[ratings["rating"] > 0]
ratings = ratings[ratings["watching_status"] != 1]
ratings = ratings.drop(columns = ["watched_episodes"])

In [9]:
def transform(df):
    df.loc[df["rating"] <= 5, ["watching_status"]] = 0.0
    df.loc[df["watching_status"] == 2, ["watching_status"]] = 1.0
    df.loc[df["watching_status"] == 4, ["watching_status"]] = 0.0
    
    df.rename(columns = {'watching_status': 'interaction'}, inplace = True)

In [10]:
transform(ratings)

In [11]:
ratings.head()

,user_id,anime_id,rating,interaction
11,0,68,6,1
12,0,1689,6,1
13,0,2913,6,1
14,0,1250,7,1
15,0,356,9,1


In [16]:
ratings = ratings[ratings["interaction"] < 2]
ratings["interaction"].value_counts()

1    51306903
0     8422493
Name: interaction, dtype: int64

In [17]:
num_users = ratings["user_id"].nunique()
num_anime = ratings["anime_id"].nunique()

print(f"There are a total of {num_users} users found.")
print(f"There are a total of {num_anime} anime found.")

print("Max user id:", ratings["user_id"].max())
print("Max anime id:", ratings["anime_id"].max())

There are a total of 310362 users found.
There are a total of 17136 anime found.
Max user id: 353404
Max anime id: 48456


If we take a look at the anime and user ids, we can notice that some values there are missing. Let's make a two helper functions, for easy conversion between dataset ids and embedding ids.

In [18]:
embId2user = sorted(ratings["user_id"].unique())
embId2anime = sorted(ratings["anime_id"].unique())

In [19]:
user2embId = {v: k for k, v in enumerate(embId2user)}
anime2embId = {v: k for k, v in enumerate(embId2anime)}

Now, let's make the dataset. It will look like a quadruples of `(user_id, anime_embId, rating, interaction)`. We'll make 3 sets: `train`, `test` and `val`.

In [20]:
np.random.seed(seed = 42)

def makeDataSet(df, split=0.95):
    n = df.to_numpy()
    
    n = np.random.permutation(n)

    x = n[:, :2]
    y = n[:, 2:]

    for i in range(x.shape[0]):
        x[i][0] = user2embId[x[i][0]]
        x[i][1] = anime2embId[x[i][1]]

    s1 = int(split * n.shape[0])
    s2 = s1 + int((1 - split) * n.shape[0] / 10)
    
    return (x[:s1], y[:s1], x[s1:s2], y[s1:s2], x[s2:], y[s2:])

In [21]:
x_train, y_train, x_test, y_test, x_val, y_val = makeDataSet(ratings)

In [22]:
y_train[:, 1].mean()

0.8589878851154062

# Models

In [23]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Neural Network

In [24]:
class HybridNeuralNetworkModel(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_dim):
        super(HybridNeuralNetworkModel, self).__init__()
        
        self.embedding_dim = embedding_dim
        
        self.user_embeddings = tf.keras.layers.Embedding(num_users, embedding_dim)
        self.item_embeddings = tf.keras.layers.Embedding(num_items, embedding_dim)

        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='relu')
        
        self.dense3 = tf.keras.layers.Dense(1, activation='sigmoid')

        self.concat = tf.keras.layers.Concatenate()
        self.dropout = tf.keras.layers.Dropout(.5)

    def call(self, inputs, training = False):
        user_ids = inputs[:, 0]
        item_ids = inputs[:, 1]

        user_embedding = self.user_embeddings(user_ids)
        item_embedding = self.item_embeddings(item_ids)

        if training:
            user_embedding = self.dropout(user_embedding, training = training)
            item_embedding = self.dropout(item_embedding, training = training)

        user_embedding = tf.reshape(user_embedding, [-1, self.embedding_dim])
        item_embedding = tf.reshape(item_embedding, [-1, self.embedding_dim])

        conc = self.concat([user_embedding, item_embedding])
        x = self.dense1(conc)
        
        x1 = self.dense2(x)
        x2 = self.dense3(x)

        return x1, x2

In [27]:
hnn_model = HybridNeuralNetworkModel(num_users = num_users, 
                                     num_items = num_anime, 
                                     embedding_dim = 64)

hnn_model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = {
        "output_1": tf.keras.losses.MeanSquaredError(),
        "output_2": tf.keras.losses.BinaryCrossentropy()
    },
    loss_weights = {
        "output_1": 1.0,
        "output_2": 5.0
    },
    metrics = {
        "output_1": [
            tf.keras.metrics.RootMeanSquaredError("RMSE")
        ],
        "output_2": [
            tf.keras.metrics.BinaryAccuracy("Acc"),
            tf.keras.metrics.Precision(name = "P"),
            tf.keras.metrics.Recall(name = "R")
        ]
    },
    run_eagerly = True
)

In [28]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)

history = hnn_model.fit(
    x = x_train, 
    y = {
        "output_1": y_train[:, 0],
        "output_2": y_train[:, 1]
    },
    batch_size = 64, 
    epochs = 100, 
    steps_per_epoch = 1000,
    callbacks = [callback],
    validation_data = (x_test, {
        "output_1": y_test[:, 0],
        "output_2": y_test[:, 1]
    }),
    validation_steps = 10,
    validation_batch_size = 64
)

Epoch 1/100
1000/1000 [==============================] - 40s 37ms/step - loss: 11.7496 - output_1_loss: 9.6836 - output_2_loss: 0.4132 - output_1_RMSE: 3.1118 - output_2_Acc: 0.8537 - output_2_P: 0.8578 - output_2_R: 0.9943 - val_loss: 4.6529 - val_output_1_loss: 2.8144 - val_output_2_loss: 0.3677 - val_output_1_RMSE: 1.6776 - val_output_2_Acc: 0.8641 - val_output_2_P: 0.8641 - val_output_2_R: 1.0000
Epoch 2/100
1000/1000 [==============================] - 37s 37ms/step - loss: 4.4881 - output_1_loss: 2.6609 - output_2_loss: 0.3655 - output_1_RMSE: 1.6312 - output_2_Acc: 0.8635 - output_2_P: 0.8651 - output_2_R: 0.9971 - val_loss: 4.5685 - val_output_1_loss: 2.7619 - val_output_2_loss: 0.3613 - val_output_1_RMSE: 1.6619 - val_output_2_Acc: 0.8656 - val_output_2_P: 0.8712 - val_output_2_R: 0.9910
Epoch 3/100
1000/1000 [==============================] - 37s 37ms/step - loss: 4.3435 - output_1_loss: 2.5452 - output_2_loss: 0.3597 - output_1_RMSE: 1.5954 - output_2_Acc: 0.8647 - output_2_P

In [29]:
hnn_model.save_weights(
    "/content/drive/Shareddrives/ML/Anime RecSys/Hybrid/NeuralNetworkModel/model", overwrite=True
)

In [ ]:
hnn_model.load_weights(
    "/content/drive/Shareddrives/ML/Anime RecSys/Hybrid/NeuralNetworkModel/model"
)

## Comparing the models

In [30]:
hnn_model.evaluate(x = x_val, y = [y_val[:, 0], y_val[:, 1]])

83995/83995 [==============================] - 1674s 20ms/step - loss: 3.5091 - output_1_loss: 1.9694 - output_2_loss: 0.3079 - output_1_RMSE: 1.4034 - output_2_Acc: 0.8790 - output_2_P: 0.8931 - output_2_R: 0.9759


[3.5090746879577637,
 1.969431757926941,
 0.30792486667633057,
 1.4033644199371338,
 0.8789761066436768,
 0.8931456804275513,
 0.9758570194244385]